In [1]:
# Making imports convenient
import sys
import os
PATH=os.getcwd().split('/notebooks')[0]
sys.path.insert(1, PATH)

import torch
from torch.utils.data import DataLoader
from datasets import load_dataset, Dataset
import transformers
from sklearn.model_selection import StratifiedKFold

from transformers import AutoTokenizer, DataCollatorWithPadding,AutoModelForSequenceClassification,TrainingArguments,Trainer

from src.utils.myutils import *
import yaml
from tqdm import tqdm
import logging
import json
import warnings
import random

logging.disable(logging.ERROR)
np.warnings.filterwarnings('ignore', category=np.VisibleDeprecationWarning)
warnings.filterwarnings("ignore", category=UserWarning) 

CS_DATA_PATH = PATH + '/data/CS/processed/BABE/train.csv'
CONFIG_PATH = PATH + '/src/utils/config.yaml'

model_name = 'fav-kky/FERNET-C5'
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False,padding=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
BATCH_SIZE = 32

KeyboardInterrupt: 

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(model_name,num_labels=2);

In [16]:
def eval_on_babe(MODEL_PATH,to_train):
    scores=[]
    print("Running 5-fold CV on model: ",model_name,"...")
    for train_index, val_index in skfold.split(token_full['input_ids'],token_full['label']):

        token_train = Dataset.from_dict(token_full[train_index])
        token_valid = Dataset.from_dict(token_full[val_index])

        model = AutoModelForSequenceClassification.from_pretrained(model_name,num_labels=2);
        if MODEL_PATH is not None:
            print("tuning pretrained")
            model.load_state_dict(torch.load(MODEL_PATH))
        model.to(device)
        if to_train:
            trainer = Trainer(model,training_args,train_dataset=token_train,data_collator=data_collator,tokenizer=tokenizer)
            trainer.train()

            #evaluation
        eval_dataloader = DataLoader(token_valid, batch_size=BATCH_SIZE, collate_fn=data_collator)
        scores.append(compute_metrics(model,device,eval_dataloader)['f1'])
        print(scores[-1])

    return np.mean(scores)

In [3]:
data = load_dataset('csv',data_files = CS_DATA_PATH)['train']
with open(CONFIG_PATH) as f:
    config_data = yaml.load(f, Loader=yaml.FullLoader)

In [4]:
skfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

In [5]:
subj_path = '/home/horyctom/bias-detection-thesis/src/models/trained/subj_balanced.pth'
mb_path = '/home/horyctom/bias-detection-thesis/src/models/trained/mb_balanced.pth'
wikinpov_path = '/home/horyctom/bias-detection-thesis/src/models/trained/wiki_balanced.pth'
all_path = '/home/horyctom/bias-detection-thesis/src/models/trained/all_balanced.pth'
wncs_path = '/home/horyctom/bias-detection-thesis/src/models/trained/wncs_balanced.pth'
all_wo_mb_path = '/home/horyctom/bias-detection-thesis/src/models/trained/all_balanced_wo_mb.pth'

In [6]:
training_args = TrainingArguments(
    output_dir = './',
    num_train_epochs=3,
    per_device_train_batch_size=BATCH_SIZE,  
    logging_steps=50,
    disable_tqdm = False,
    save_total_limit=2,
    weight_decay=0.1,
    learning_rate=2e-5)

In [7]:
token_full = preprocess_data(data,tokenizer,'sentence')

## Vanilla

In [19]:
eval_on_babe(None)

Running 5-fold CV on model:  fav-kky/FERNET-C5 ...


Step,Training Loss
50,0.591600
100,0.486800
150,0.394800
200,0.356300
250,0.292800


0.7749219263642341


Step,Training Loss
50,0.587500
100,0.510500
150,0.434000
200,0.391800
250,0.312700


0.817824430988533


...

Step,Training Loss
50,0.591300
100,0.509800
150,0.418900
200,0.386800
250,0.321900


0.7900947057910263


Step,Training Loss
50,0.566300
100,0.476500
150,0.371600
200,0.349300
250,0.286400


0.7776377627433766


0.7813925882103944

In [20]:
print(0.7813925882103944)

0.7813925882103944


## SUBJ

In [15]:
eval_on_babe(subj_path)

Running 5-fold CV on model:  fav-kky/FERNET-C5 ...


Step,Training Loss
50,0.547500
100,0.474200
150,0.373900
200,0.326200
250,0.279100


0.7684911242603552


Step,Training Loss
50,0.587800
100,0.491300
150,0.397400
200,0.371500
250,0.286700


0.8115015974440895


...

Step,Training Loss
50,0.555900
100,0.480500
150,0.373400
200,0.352100
250,0.281700


0.7769188522636601


Step,Training Loss
50,0.553400
100,0.457800
150,0.364600
200,0.337400
250,0.278200


0.7747152995543819


0.7877574484554553

In [17]:
print(0.7877574484554553)

0.7877574484554553


## WIKI

In [21]:
eval_on_babe(wikinpov_path)

Running 5-fold CV on model:  fav-kky/FERNET-C5 ...
tuning pretrained


Step,Training Loss
50,0.505200
100,0.455400
150,0.354700
200,0.314900
250,0.254500


0.7722329838370794
tuning pretrained


Step,Training Loss
50,0.532700
100,0.464800
150,0.374000
200,0.348900
250,0.264900


0.8178616275152368
tuning pretrained


...

Step,Training Loss
50,0.504000
100,0.472100
150,0.369000
200,0.344400
250,0.273800


0.7704527320394184
tuning pretrained


Step,Training Loss
50,0.528500
100,0.456200
150,0.365200
200,0.326200
250,0.266600


0.7976091187100363


0.7841230297009795

In [22]:
print(0.7841230297009795)

0.7841230297009795


## MB

In [23]:
eval_on_babe(mb_path)

Running 5-fold CV on model:  fav-kky/FERNET-C5 ...
tuning pretrained


Step,Training Loss
50,0.535600
100,0.466800
150,0.348600
200,0.295800
250,0.235500


0.7575222335843326
tuning pretrained


Step,Training Loss
50,0.522300
100,0.487700
150,0.364900
200,0.334000
250,0.249400


0.7950564740546733
tuning pretrained


...

Step,Training Loss
50,0.531600
100,0.483700
150,0.377500
200,0.326200
250,0.263700


0.7695356797303343
tuning pretrained


Step,Training Loss
50,0.539600
100,0.473400
150,0.345100
200,0.313700
250,0.241700


0.7616353499896757


0.7703332626778344

In [24]:
print(0.7703332626778344)

0.7703332626778344


## All

In [25]:
eval_on_babe(all_path)

Running 5-fold CV on model:  fav-kky/FERNET-C5 ...
tuning pretrained


Step,Training Loss
50,0.522100
100,0.469200
150,0.346700
200,0.306100
250,0.252200


0.7848797250859105
tuning pretrained


Step,Training Loss
50,0.539700
100,0.477600
150,0.372500
200,0.352900
250,0.268900


0.8337826797385621
tuning pretrained


...

Step,Training Loss
50,0.512500
100,0.475300
150,0.357500
200,0.335100
250,0.250800


0.7636927673967402
tuning pretrained


Step,Training Loss
50,0.525000
100,0.448200
150,0.360300
200,0.322700
250,0.261500


0.7653289372005565


0.784179272671834

In [26]:
print(0.784179272671834)

0.784179272671834


## WNCS

In [27]:
eval_on_babe(wncs_path)

Running 5-fold CV on model:  fav-kky/FERNET-C5 ...
tuning pretrained


Step,Training Loss
50,0.528400
100,0.454500
150,0.360800
200,0.320400
250,0.281700


0.7663056262976486
tuning pretrained


Step,Training Loss
50,0.536000
100,0.473000
150,0.385300
200,0.340900
250,0.271500


0.8144673370942687
tuning pretrained


...

Step,Training Loss
50,0.537200
100,0.465400
150,0.376700
200,0.348500
250,0.289900


0.7673570836785418
tuning pretrained


Step,Training Loss
50,0.548800
100,0.460800
150,0.374900
200,0.322600
250,0.278400


0.7743055555555556


0.7815197745890112

In [28]:
print(0.7815197745890112)

0.7815197745890112


## All wo MB

In [8]:
eval_on_babe(all_wo_mb_path)

Running 5-fold CV on model:  fav-kky/FERNET-C5 ...
tuning pretrained


Step,Training Loss
50,0.518000
100,0.459500
150,0.352600
200,0.308000
250,0.250700


0.7724102616621435
tuning pretrained


Step,Training Loss
50,0.570300
100,0.464200
150,0.374100
200,0.340600
250,0.267700


0.8080696590630365
tuning pretrained


...

Step,Training Loss
50,0.517600
100,0.474000
150,0.365400
200,0.347200
250,0.266400


0.7875887110084172
tuning pretrained


Step,Training Loss
50,0.523300
100,0.449200
150,0.354100
200,0.315300
250,0.255400


0.7588995930142703


0.7844111431620024

In [9]:
print(0.7844111431620024)

0.7844111431620024


## Eval all on BABE not fine-tuning

In [18]:
eval_on_babe(subj_path,False)

0.5542269457401406


In [20]:
eval_on_babe(wikinpov_path,False)

0.634449005739866


In [22]:
eval_on_babe(mb_path,False)

0.46307524184646615


In [24]:
eval_on_babe(all_path,False)

0.6422974189680117


In [26]:
eval_on_babe(wncs_path,False)

0.6697388712101715


In [28]:
eval_on_babe(all_wo_mb_path,False)

0.5280019026747172
